# Reducing Optimization Risk through Portfolio Methods

  * John Mount, Win-Vector LLC
  * August 11, 2025


## Introduction

I want to discuss how fragile optimization solutions to real world problems can be. And how to solve that.

Small changes in modeling strategy, assumptions, data, estimates, constraints, or objective can lead to unstable and degenerate solutions. To warm up let's discuss one of the most famous optimization examples: Stigler's minimal subsistence diet problem.

George B. Dantzig famously used Stigler's minimal subsistence diet problem as an early example for linear programming. He later commented on this and the fragility and undesirability of naive optimization solutions in George B. Dantzig, "The Diet Problem", Interfaces, Vol. 20, No. 4, The Practice of Mathematical Programming (Jul. - Aug., 1990), pp. 43-47. The source of the model and diet data: George J. Stigler, "The Cost of Subsistence", Journal of Farm Economics, May, 1945, Vol. 27, No. 2 (May, 1945), pp. 303-314 is itself fairly remarkable. 

The original Stigler writeup in fact seems hostile to the minimal subsistence diet problem and includes the following:

  * Previous claimed low-cost solutions in fact "cost about two or three times as much as a [true] minimum cost diet."
  * "The BLS [Bureau of Labor Statistics] list [which the diet is based on] is a short one , and it excludes almost all fresh fruits, nuts, many cheap vegetables rich in nutrients and fresh fish."
  * "No one recommends these diets for anyone, let alone everyone; it would be the height of absurdity to practice extreme economy at the dinner table in order to have an excess of housing or recreation or leisure.'

That being said, let's take a look at this famous minimal subsistence diet.


## The Original diet

We start with the 1939 cost and nutritional content table from Stigler's paper. It consisted of 77 food items with the amount of nutrients per penny as feature columns. This copy of the data is from https://developers.google.com/optimization/lp/stigler_diet . All of the Python code and data that produced this article can be found [here](https://github.com/WinVector/Examples/blob/main/OptimizationRisk/Stigler_diet.ipynb).

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown
from diet_fns import *

In [2]:
data

,item,unit,price,Calories (kcal),Protein (g),Calcium (g),Iron (mg),Vitamin A (KIU),Vitamin B1 (mg),Vitamin B2 (mg),Niacin (mg),Vitamin C (mg)
0,Wheat Flour (Enriched),10 lb.,36.0,44.7,1411,2.0,365,0.0,55.4,33.3,441,0
1,Macaroni,1 lb.,14.1,11.6,418,0.7,54,0.0,3.2,1.9,68,0
2,Wheat Cereal (Enriched),28 oz.,24.2,11.8,377,14.4,175,0.0,14.4,8.8,114,0
3,Corn Flakes,8 oz.,7.1,11.4,252,0.1,56,0.0,13.5,2.3,68,0
4,Corn Meal,1 lb.,4.6,36.0,897,1.7,99,30.9,17.4,7.9,106,0
...,...,...,...,...,...,...,...,...,...,...,...,...
72,Chocolate,8 oz.,16.2,8.0,77,1.3,39,0.0,0.9,3.4,14,0
73,Sugar,10 lb.,51.7,34.9,0,0.0,0,0.0,0.0,0.0,0,0
74,Corn Syrup,24 oz.,13.7,14.7,0,0.5,74,0.0,0.0,0.0,5,0
75,Molasses,18 oz.,13.6,9.0,0,10.3,244,0.0,1.9,7.5,146,0


 

The columns are:

  * `item` name of the food item. This is the row key.
  * `unit` what the colloquial unit for the commodity was. I think that was left in as a complaint about the data sources. The author pre-processed the data so that we do not have to use this column.
  * `price` the price of the standard amount of the item in 1939 in US cents. The author pre-processed the data so that we do not have to use this column.
  * All other columns: the amount of the given nutrient to be had for one penny's worth of the food item. The author prepared this column by taking the price through the units column and using references for the expected nutritional loading of the food item in question. Note: the original author did not consider these numbers to be reliable.

We also have, also from Stigler, approximate estimated minimum daily nutritional needs.

In [3]:
nutrients

,requirement,lower bound
0,Calories (kcal),3.0
1,Protein (g),70.0
2,Calcium (g),0.8
3,Iron (mg),12.0
4,Vitamin A (KIU),5.0
5,Vitamin B1 (mg),1.8
6,Vitamin B2 (mg),2.7
7,Niacin (mg),18.0
8,Vitamin C (mg),75.0


Stigler set this up as minimizing a linear function (sum of costs) subject to linear constraints (meeting all of the nutritional minimums). Dantzig famously re-solved this using the simplex method, getting a solution value quite close to Stigler's heuristic solution.

We use Python to reproduce the Dantzig solution below.

In [4]:
soln = solve_diet(data=data, nutrients=nutrients)
selected = soln > 1e-6
selected_df = data.loc[selected, :].reset_index(drop=True, inplace=False)
selected_df["spend"] = np.array(soln)[selected]
selected_df["fraction of spend"] = selected_df["spend"] / np.sum(selected_df["spend"])

selected_df.round(4)

,item,unit,price,Calories (kcal),Protein (g),Calcium (g),Iron (mg),Vitamin A (KIU),Vitamin B1 (mg),Vitamin B2 (mg),Niacin (mg),Vitamin C (mg),spend,fraction of spend
0,Wheat Flour (Enriched),10 lb.,36.0,44.7,1411,2.0,365,0.0,55.4,33.3,441,0,0.0295,0.2717
1,Liver (Beef),1 lb.,26.8,2.2,333,0.2,139,169.2,6.4,50.8,316,525,0.0019,0.0174
2,Cabbage,1 lb.,3.7,2.6,125,4.0,36,7.2,9.0,4.5,26,5369,0.0112,0.1032
3,Spinach,1 lb.,8.1,1.1,106,0.0,138,918.4,5.7,13.8,33,2755,0.0050,0.0461
4,"Navy Beans, Dried",1 lb.,5.9,26.9,1691,11.4,792,0.0,38.4,24.6,217,0,0.0610,0.5616


In [5]:
# 1939 not a leap year
display(
    Markdown(
        f"""
Cost for full year in 1939: \\${365.0 * np.sum(soln):0.2f}
"""
    )
)


Cost for full year in 1939: \$39.66


And this matches Dantzig's answer. The diet is essentially Navy Beans plus Wheat Flour, with 3 other items patching up nutritional deficiencies. We could enforce diversity and risk reduction by adding a per-item upper bounds on spend or weight. Or, as we pursue in this note, we could also directly model risk.

The next table confirms that expected value of the diet meets or exceeds the lower bounds.

In [6]:
build_check_frame(soln, data=data, nutrients=nutrients).round(2)

,requirement,lower bound,expected value
0,Calories (kcal),3.0,3.00
1,Protein (g),70.0,147.41
2,Calcium (g),0.8,0.80
3,Iron (mg),12.0,60.47
4,Vitamin A (KIU),5.0,5.00
5,Vitamin B1 (mg),1.8,4.12
6,Vitamin B2 (mg),2.7,2.70
7,Niacin (mg),18.0,27.32
8,Vitamin C (mg),75.0,75.00


Obviously this is a fairly miserable unvarying diet. Such a plan is likely very fragile and unstable. Also, a small change in needs, price, or nutrition may completely change which diets are considered acceptable.

In fact this diet violates Akin's 8th law of spacecraft design (source: [Akin's Laws of Spacecraft Design](https://www.ece.uvic.ca/~elec399/201409/Akin%27s%20Laws%20of%20Spacecraft%20Design.pdf)):


<blockquote>
"In nature, the optimum is almost always in the middle somewhere. Distrust assertions that the optimum is at an extreme point."
</blockquote>

## Dantzig's later diet

Dantzig himself later went on a weight loss diet, this time planned on Rand's IBM 701 computer. His idea was to maximize the unit weight of food less the weight of its water content from over 500 foods entered as punched cards. As is often the case: the optimizer ["cheesed"](https://www.inverse.com/gaming/how-street-fighter-invented-gamings-most-despised-strategy?utm_source=chatgpt.com) and found data errors and corner cases instead of producing an interesting solution. Daily suggested diets included:

  * 500 gallons of vinegar, as the data mis-entered vinegar as having no water content.
  * 200 bouillon cubes, which indeed have a low proportion of water.
  * A diet including 2 pounds of bran per day.
  * The same diet with the bran replaced with blackstrap molasses.

Dantzig stopped after this, and claims this experience was "how upper bounds on variables in linear programming first began."

Dantzig's experience gets repeated again and again. One of the graces of linear programs is that at the solution usually only a few of the inequality constraints are "active" or at equality at an optimal solution. However, conversely, if there are any modeling errors- these usually volunteer to be the active constraints. We have been researching some methods to mitigate these issues, some of which we discuss here and some of which we will discuss in a later note.

Optimization is incredibly useful, it just takes a bit of care to work well with uncertain data.

## One has to move beyond totals

When planning under uncertainty one must move beyond maximizing total expected value. One has to deal with risk and error. We are going to mitigate risk using standard portfolio theory methods.

The risk model we are going to used is based on the following example.

Both corn flakes and macaroni are recorded as supplying 68mg of niacin per penny. Suppose we are choosing between the following 3 plans: 2 cents worth of corn flakes, 2 cents worth of macaroni, or 1 cent each of corn flakes and macaroni.

<img src="plans.jpg" width="600"  height="450">

Let's model the unknown true value of each of these food items as having 48 or 88 mg of niacin with equal chance. We are modeling this as uncertainty in our estimate of 68 mg of niacin, and not uncertainty in what is delivered. That is: two servings of corn flakes will have identical amounts of niacin. Let's assume the same independently for macaroni.

Then:

  * Plan 1 is worth 48 + 48 or 88 + 88 with equal probability. This an expected value of 136mg and a (population) standard deviation of 40.0mg.
  * Plan 2 is worth 48 + 48 or 88 + 88 with equal probability. This an expected value of 136mg and a (population) standard deviation of 40.0mg.
  * Plan 3 is worth one of 48 + 48, 48 + 88, 88 + 48, or 88 + 88 with equal probability. This an expected value of 136mg and a (population) standard deviation of 28.3mg.

The argument is: plan 3 is preferable as it has an expected value no lower than the other plans and a standard deviation strictly lower than the other plans. It is a quantified version of not putting all of one's eggs in the same basket.

In the next section we systematize this sort of risk adjustment or pricing.

## Modeling uncertainty

Dantzig, "Linear Programming and Extensions", Princeton University Press, 1963 treats dealing with uncertainty in Chapter 25. The idea is to model variance and pick a best plan subject to a variance bound (ala. Markowitz portfolio theory). Ideas like this can improve the utility and reliability of optimization solutions.

To get the improvement we need to introduce an uncertainty model. A very simple one is:

  * Use a mean/variance modeling discipline.
  * Treat adding more of the same foods to a diet as a re-scaling. That is: as mean scales linearly, variance scales by a squared factor.
  * Treat different foods in a diet as independent random variables. That is: means add and variances also add (which is slower than the previous line's square scaling).

This is what was going on in our cornflakes and macaroni example.

We can illustrate the scoring method with the following simple made up diet set of requirements. We add a new column to our requirements table: "coefficient of variation". This is the ratio of standard-deviation to total (or essentially one over the Sharpe ratio). In our case the column models how uncertain we are on the bound. For now we just put in 0.25 as this uncertainty parameter. This parameter will end up mattering only up to scale, so all we are saying is: we don't yet have any data to facilitate treating any of the nutrition bounds as having different uncertainty. We are also not modeling uncertainty in price or in per item nutrient values, though the same methodology would support this.

In [7]:
requirements_f = pd.DataFrame({"requirement": [f"nutrient_{j}" for j in range(1, 6)]})
requirements_f["lower bound"] = 200
requirements_f["coefficient of variation"] = 0.25

requirements_f

,requirement,lower bound,coefficient of variation
0,nutrient_1,200,0.25
1,nutrient_2,200,0.25
2,nutrient_3,200,0.25
3,nutrient_4,200,0.25
4,nutrient_5,200,0.25


And next is a simple list of items. 

In [8]:
example_f = pd.DataFrame(
    {
        "item": [f"item_{i}" for i in range(1, requirements_f.shape[0] + 1)],
    }
)
for j, nm in enumerate(requirements_f["requirement"]):
    example_f[nm] = (
        1000.0
        * np.array([1] * (j + 1) + [0] * (example_f.shape[0] - (j + 1)))
        / (j + 1)
    )
example_f["spend"] = 1.0 / example_f.shape[0]

example_f

,item,nutrient_1,nutrient_2,nutrient_3,nutrient_4,nutrient_5,spend
0,item_1,1000.0,500.0,333.333333,250.0,200.0,0.2
1,item_2,0.0,500.0,333.333333,250.0,200.0,0.2
2,item_3,0.0,0.0,333.333333,250.0,200.0,0.2
3,item_4,0.0,0.0,0.000000,250.0,200.0,0.2
4,item_5,0.0,0.0,0.000000,0.0,200.0,0.2


The spend column indicates we are spending 0.2 cents on each item which gives us 1000 * 0.2 = 200 units of each nutrient. Notice we take 200 units of nutrient `k` from `k` different items. We use this to show picking up nutrients in lowe to high diversity manners.

When we inspect this proposed problem and solution we see the following.

In [9]:
cf = build_check_frame(
    soln=example_f["spend"],
    data=example_f,
    nutrients=requirements_f,
    risk_scale=1,
    do_assert=False,
)

cf.round(2)

,requirement,lower bound,expected value,modeled risk,risk adjusted expectation
0,nutrient_1,200,200.0,50.00,150.00
1,nutrient_2,200,200.0,35.36,164.64
2,nutrient_3,200,200.0,28.87,171.13
3,nutrient_4,200,200.0,25.00,175.00
4,nutrient_5,200,200.0,22.36,177.64


Notice we meet the 200 lower bound for each nutrient. However, we are modeling larger risk for nutrients that are coming from fewer sources. We define a user parameter encoding our risk aversion called "`risk_scale`". Then the `risk adjusted expectation` is defined as the expected value minus `risk_scale` standard deviations. Note: this is a common heuristic that follows from mean/variance risk analysis, *and* what is entailed in multi-objective optimization. Some notes on this formulation can be found [here](https://win-vector.com/2025/08/08/modeling-uncertainty-for-better-outcomes/).

## Our last solution

Our last solution met lower bounds, but not with risk adjusted expectations.

Let's evaluate the old solution with a (somewhat arbitrary) `risk_scale` of `1`. `risk_scale` is a user supplied assumed sensitivity to risk. To get this value we have to try a few values (such as `1`) and see how this changes the outcomes. We add a (somewhat arbitrary) `coefficient of variation` to our nutrients and model dependent and independent combinations of foods as outline above.


In [10]:
nutrients["coefficient of variation"] = 0.25
risk_scale = 1.0


nutrients

,requirement,lower bound,coefficient of variation
0,Calories (kcal),3.0,0.25
1,Protein (g),70.0,0.25
2,Calcium (g),0.8,0.25
3,Iron (mg),12.0,0.25
4,Vitamin A (KIU),5.0,0.25
5,Vitamin B1 (mg),1.8,0.25
6,Vitamin B2 (mg),2.7,0.25
7,Niacin (mg),18.0,0.25
8,Vitamin C (mg),75.0,0.25


In [11]:
build_check_frame(
    soln, data=data, nutrients=nutrients, risk_scale=1, do_assert=False
).round(2)

,requirement,lower bound,expected value,modeled risk,risk adjusted expectation
0,Calories (kcal),3.0,3.00,0.53,2.47
1,Protein (g),70.0,147.41,27.82,119.59
2,Calcium (g),0.8,0.80,0.17,0.63
3,Iron (mg),12.0,60.47,12.38,48.08
4,Vitamin A (KIU),5.0,5.00,1.15,3.85
5,Vitamin B1 (mg),1.8,4.12,0.71,3.41
6,Vitamin B2 (mg),2.7,2.70,0.45,2.25
7,Niacin (mg),18.0,27.32,4.65,22.67
8,Vitamin C (mg),75.0,75.00,15.44,59.56


We see that our risk adjusted expectations did not exceed the lower bounds. To deal with this we can either risk-inflate the lower bounds, or risk discount the computed expected values. For independent risk (as we are modeling) here the two methods should give similar solutions, and the bound inflation is easier to implement. We will explore expectation discounting as it could later be expanded to deal with correlated risks.

## Risk discounts in action

Let's see this in action. We re-solve for the diet for a small `risk_scale` using a quadratic program solver.  

For each `risk_scale` the modeler specifies, we get a new solution. Below is the solution for `risk_scale = 1.0` (the common "1 standard deviation below estimated expected value" heuristic).

In [12]:
soln_r = solve_diet(data=data, nutrients=nutrients, risk_scale=risk_scale)
selected = soln_r > 1e-6
selected_df = data.loc[selected, :].reset_index(drop=True, inplace=False)
selected_df["spend"] = np.array(soln_r)[selected]
selected_df["fraction of spend"] = selected_df["spend"] / np.sum(selected_df["spend"])

selected_df.round(4)

,item,unit,price,Calories (kcal),Protein (g),Calcium (g),Iron (mg),Vitamin A (KIU),Vitamin B1 (mg),Vitamin B2 (mg),Niacin (mg),Vitamin C (mg),spend,fraction of spend
0,Wheat Flour (Enriched),10 lb.,36.0,44.7,1411,2.0,365,0.0,55.4,33.3,441,0,0.0431,0.3249
1,Wheat Cereal (Enriched),28 oz.,24.2,11.8,377,14.4,175,0.0,14.4,8.8,114,0,0.0003,0.0025
2,Evaporated Milk (can),14.5 oz.,6.7,8.4,422,15.1,9,26.0,3.0,23.5,11,60,0.0059,0.0448
3,Cheese (Cheddar),1 lb.,24.2,7.4,448,16.4,19,28.1,0.8,10.3,4,0,0.0043,0.0321
4,Cabbage,1 lb.,3.7,2.6,125,4.0,36,7.2,9.0,4.5,26,5369,0.0141,0.1062
5,Spinach,1 lb.,8.1,1.1,106,0.0,138,918.4,5.7,13.8,33,2755,0.0067,0.0506
6,"Navy Beans, Dried",1 lb.,5.9,26.9,1691,11.4,792,0.0,38.4,24.6,217,0,0.0582,0.4389


In [13]:
display(
    Markdown(
        f"""
Cost for full year in 1939: \\${365.0 * np.sum(soln_r):0.2f}.
"""
    )
)


Cost for full year in 1939: \$48.40.


The diet is a a bit more diverse, at a moderate increase in cost.

As we see in the next table: the risk adjustment ensures that the risk adjusted estimates of the expected nutrition are not below the requirements. This is done both by adding a bit of a buffer and preferring a more diverse diet.

In [14]:
build_check_frame(soln_r, data=data, nutrients=nutrients, risk_scale=risk_scale).round(
    2
)

,requirement,lower bound,expected value,modeled risk,risk adjusted expectation
0,Calories (kcal),3.0,3.62,0.62,3.00
1,Protein (g),70.0,166.22,28.93,137.28
2,Calcium (g),0.8,0.97,0.17,0.80
3,Iron (mg),12.0,63.45,12.18,51.27
4,Vitamin A (KIU),5.0,6.54,1.54,5.00
5,Vitamin B1 (mg),1.8,4.81,0.82,3.99
6,Vitamin B2 (mg),2.7,3.21,0.51,2.70
7,Niacin (mg),18.0,32.34,5.70,26.63
8,Vitamin C (mg),75.0,94.46,19.46,75.00


## With a larger risk parameter

We can run this again with a higher risk parameter such as `risk_scale = 2`. The `risk_scale` is a modeling hyper-parameter that we have to try a few values of and judge results to pick a best for the situation solution.

This gives us a more diverse diet, at a somewhat higher expense, with more of a safety buffer in the nutrient requirements, and hopefully lower the risk brought in by uncertainty in the value of any one ingredient.

In [15]:
risk_scale = 2

In [16]:
soln_rlarge = solve_diet(data=data, nutrients=nutrients, risk_scale=risk_scale)

In [17]:
selected = soln_rlarge > 1e-6
selected_df = data.loc[selected, :].reset_index(drop=True, inplace=False)
selected_df["spend"] = np.array(soln_rlarge)[selected]
selected_df["fraction of spend"] = selected_df["spend"] / np.sum(selected_df["spend"])

selected_df.round(4)

,item,unit,price,Calories (kcal),Protein (g),Calcium (g),Iron (mg),Vitamin A (KIU),Vitamin B1 (mg),Vitamin B2 (mg),Niacin (mg),Vitamin C (mg),spend,fraction of spend
0,Wheat Flour (Enriched),10 lb.,36.0,44.7,1411,2.0,365,0.0,55.4,33.3,441,0,0.0401,0.2444
1,Wheat Cereal (Enriched),28 oz.,24.2,11.8,377,14.4,175,0.0,14.4,8.8,114,0,0.0039,0.0239
2,Corn Meal,1 lb.,4.6,36.0,897,1.7,99,30.9,17.4,7.9,106,0,0.0106,0.0646
3,Evaporated Milk (can),14.5 oz.,6.7,8.4,422,15.1,9,26.0,3.0,23.5,11,60,0.0176,0.1073
4,Cheese (Cheddar),1 lb.,24.2,7.4,448,16.4,19,28.1,0.8,10.3,4,0,0.0043,0.0265
5,Lard,1 lb.,9.8,41.7,0,0.0,0,0.2,0.0,0.5,5,0,0.0039,0.0236
6,Cabbage,1 lb.,3.7,2.6,125,4.0,36,7.2,9.0,4.5,26,5369,0.0110,0.0672
7,Potatoes,15 lb.,34.0,14.3,336,1.8,118,6.7,29.4,7.1,198,2522,0.0096,0.0586
8,Spinach,1 lb.,8.1,1.1,106,0.0,138,918.4,5.7,13.8,33,2755,0.0069,0.0420
9,Sweet Potatoes,1 lb.,5.1,9.6,138,2.7,54,290.7,8.4,5.4,83,1912,0.0027,0.0162


In [18]:
display(
    Markdown(
        f"""
Cost for full year in 1939: \\${365.0 * np.sum(soln_rlarge):0.2f}.
"""
    )
)


Cost for full year in 1939: \$59.83.


In [19]:
build_check_frame(
    soln_rlarge, data=data, nutrients=nutrients, risk_scale=risk_scale
).round(2)

,requirement,lower bound,expected value,modeled risk,risk adjusted expectation
0,Calories (kcal),3.0,4.09,1.09,3.00
1,Protein (g),70.0,165.96,46.54,119.42
2,Calcium (g),0.8,1.08,0.28,0.80
3,Iron (mg),12.0,57.89,18.56,39.33
4,Vitamin A (KIU),5.0,8.20,3.20,5.00
5,Vitamin B1 (mg),1.8,4.89,1.40,3.49
6,Vitamin B2 (mg),2.7,3.60,0.90,2.70
7,Niacin (mg),18.0,32.34,10.05,22.30
8,Vitamin C (mg),75.0,108.42,33.42,75.00


## Conclusion

Putting even a crude dependency model on top of the diet problem allows one to pick more robust interior solutions. One can even use human intuition to pick from a small set of proposed "efficient frontier" diets.

Adding a portfolio uncertainty model doesn't magically fix everything. For example, the above model still has no direct notions of recipes, choice, palatability, status, cultural norms, or variety. However it does help prevent your optimizer from making your approximate objective function and approximate constraints appear worse then they really are. Without regularization or uncertainty optimizers routinely search for embarrassing exceptions instead of solving the actual problem.

Uncertainty is routinely neglected as it requires a more powerful optimizer. Tools like this were not available in the 1940s. But the point is: we now have access to such optimizers and access to people that know how to use them.